# Solving the knapsack problem in 3 ways

1. brute force
2. dynamic programming
3. recursion

In [7]:
from itertools import combinations


def brute_force(items, capacity):
    # https://rosettacode.org/wiki/Knapsack_problem/0-1#Brute_force_algorithm
    def anycomb(items):
        "return combinations of any length from the items"
        return (comb for r in range(1, len(items) + 1) for comb in combinations(items, r))

    def totalvalue(comb):
        "Totalise a particular combination of items"
        totwt = totval = 0
        for item, wt, val in comb:
            totwt += wt
            totval += val
        return (totval, -totwt) if totwt <= capacity else (0, 0)

    bagged = max(anycomb(items), key=totalvalue)  # max val or min wt if values equal
    val, wt = totalvalue(bagged)
    return val, -wt, bagged

In [8]:
def dynamic_programming(items, capacity):
    # https://rosettacode.org/wiki/Knapsack_problem/0-1#Dynamic_programming_solution

    def totalvalue(comb):
        "Totalise a particular combination of items"
        totwt = totval = 0
        for item, wt, val in comb:
            totwt += wt
            totval += val
        return (totval, -totwt) if totwt <= 400 else (0, 0)

    def knapsack01_dp(items, limit):
        table = [[0 for w in range(limit + 1)] for j in range(len(items) + 1)]

        for j in range(1, len(items) + 1):
            item, wt, val = items[j - 1]
            for w in range(1, limit + 1):
                if wt > w:
                    table[j][w] = table[j - 1][w]
                else:
                    table[j][w] = max(table[j - 1][w], table[j - 1][w - wt] + val)

        result = []
        w = limit
        for j in range(len(items), 0, -1):
            was_added = table[j][w] != table[j - 1][w]

            if was_added:
                item, wt, val = items[j - 1]
                result.append(items[j - 1])
                w -= wt

        return result

    bagged = knapsack01_dp(items, capacity)
    val, wt = totalvalue(bagged)
    return val, -wt, bagged

In [9]:

def recursion(items, capacity):
    # https://rosettacode.org/wiki/Knapsack_problem/0-1#Recursive_dynamic_programming_algorithm
    def total_value(items, max_weight):
        return sum([x[2] for x in items]) if sum([x[1] for x in items]) <= max_weight else 0

    cache = {}

    def solve(items, max_weight):
        if not items:
            return ()
        if (items, max_weight) not in cache:
            head = items[0]
            tail = items[1:]
            include = (head,) + solve(tail, max_weight - head[1])
            dont_include = solve(tail, max_weight)
            if total_value(include, max_weight) > total_value(dont_include, max_weight):
                answer = include
            else:
                answer = dont_include
            cache[(items, max_weight)] = answer
        return cache[(items, max_weight)]

    solution = solve(items, capacity)
    val = total_value(solution, capacity)
    wt = sum([x[1] for x in solution])
    return val, wt, solution

In [10]:
def solve(items, capacity, method="dp"):
    methods = {"dp": dynamic_programming, "bf": brute_force, "r": recursion}
    m = methods.get(method)
    return m(items, capacity)

In [11]:
items = (
        ("map", 9, 150),
        ("compass", 13, 35),
        ("water", 153, 200),
        ("sandwich", 50, 160),
        ("glucose", 15, 60),
        ("tin", 68, 45),
        ("banana", 27, 60),
        ("apple", 39, 40),
        ("cheese", 23, 30),
        ("beer", 52, 10),
        ("suntan cream", 11, 70),
        ("camera", 32, 30),
        ("t-shirt", 24, 15),
        ("trousers", 48, 10),
        ("umbrella", 73, 40),
        ("waterproof trousers", 42, 70),
        ("waterproof overclothes", 43, 75),
        ("note-case", 22, 80),
        ("sunglasses", 7, 20),
        ("towel", 18, 12),
        ("socks", 4, 50),
        ("book", 30, 10),
    )
capacity = 400

import time

a = time.perf_counter()
val1, wt1, items1 = solve(items, capacity, "bf")
b = time.perf_counter()
print(round(b - a, 6), "s", brute_force.__name__)
val2, wt2, items2 = solve(items, capacity, "dp")
c = time.perf_counter()
print(round(c - b, 6), "s", dynamic_programming.__name__)
val3, wt3, items3 = solve(items, capacity, "r")
d = time.perf_counter()
print(round(d - c, 6), "s", recursion.__name__)
assert val1 == val2 == val3
assert wt1 == wt2 == wt3
assert sorted(items1) == sorted(items2) == sorted(items3)

3.304391 s brute_force
0.002093 s dynamic_programming
0.01933 s recursion
